In [2]:
import tarfile
import os

Extracting Prometheus metrics for tarball

In [4]:
def extract_tarball(tarfile_path, tarfile_target):
    file = tarfile.open(tarfile_path)
    for filename in file.getnames():
        print(filename)
    file.extractall(tarfile_target)
    file.close()

In [5]:
tarfile_path = "../../prometheus/metrics/mahti_metrics_110423-210423.tgz"
tarfile_target = "./mahti_metrics_110423-210423"

In [ ]:
# Extracting tarballfile = tarfile.open(tarfile_path)
print("Extracting files:")
extract_tarball(tarfile_path, tarfile_target)

In [51]:
metric_names = []
partition_names = []
metrics = {}

def getHeader(metrics_names):
    header = "date," + ",".join(metric_names) + '\n'
    return header

def read_metrics(metrics_file, metrics):
    with open(metrics_file, 'r') as f:
        lines = f.readlines()
        for line in lines[1:]:
            line_split = line.split(',')
            metric_name = line_split[0]
            metric_name = metric_name.replace('slurm_partition_', '')
            if metric_name not in metric_names:
                metric_names.append(metric_name)
            partition = line_split[-1].rstrip()
            if partition not in metrics:
                metrics[partition] = {}
            if metric_name not in metrics[partition]:
                metrics[partition][metric_name] = {}
            index = 2
            for split in line_split[1::2]:
                if '[' in split or ']' in split:
                    # Collect metrics
                    timestamp = split[2:]
                    metric = line_split[index][0:-2].lstrip()
                    index += 2
                    metrics[partition][metric_name][timestamp] = metric
    

For each metric file, process metrics in file
Process partition metrics
TODO: Process job metrics
TODO: Generate metrics cvs

In [53]:
for root, dirs, files in os.walk(tarfile_target):
    tarfile_target = root   
    for file in files:
        if not file.startswith("slurm_partition_job"):
            tarfile_path = tarfile_target + '/' + file
            print("Extracting: " + tarfile_path)
            # unzip file tarball
            extract_tarball(tarfile_path, tarfile_target)
            # read metrics
            metrics_file = tarfile_target + '/' + file.replace(".tgz", "")
            read_metrics(metrics_file, metrics)
            # remove unzip tarball
            os.remove(metrics_file)


Extracting: ./mahti_metrics_110423-210423/04-21-2023_10-54-01/slurm_partition_availability_1w.csv.tgz
slurm_partition_availability_1w.csv
Extracting: ./mahti_metrics_110423-210423/04-21-2023_10-54-01/slurm_partition_avg_alloc_mem_1w.csv.tgz
slurm_partition_avg_alloc_mem_1w.csv
Extracting: ./mahti_metrics_110423-210423/04-21-2023_10-54-01/slurm_partition_avg_allocated_cpus_per_job_1w.csv.tgz
slurm_partition_avg_allocated_cpus_per_job_1w.csv
Extracting: ./mahti_metrics_110423-210423/04-21-2023_10-54-01/slurm_partition_avg_allocated_nodes_per_job_1w.csv.tgz
slurm_partition_avg_allocated_nodes_per_job_1w.csv
Extracting: ./mahti_metrics_110423-210423/04-21-2023_10-54-01/slurm_partition_avg_cpus_load_lower_1w.csv.tgz
slurm_partition_avg_cpus_load_lower_1w.csv
Extracting: ./mahti_metrics_110423-210423/04-21-2023_10-54-01/slurm_partition_avg_cpus_load_upper_1w.csv.tgz
slurm_partition_avg_cpus_load_upper_1w.csv
Extracting: ./mahti_metrics_110423-210423/04-21-2023_10-54-01/slurm_partition_avg_ex

In [64]:
if not os.path.exists("partitions"):
    os.makedirs("partitions")

for partition in metrics.keys(): # Partition iterator
    filename = partition + '.csv'
    print("Creating " + filename)
    with open('partitions/' + filename, 'w') as w:
        # Write header
        metric_names = metrics[partition].keys()
        header = getHeader(metric_names)
        w.write(header)
        # Collect metrics
        partition_metrics = {}
        for metric_name in metrics[partition].keys(): # Metrics iterator
            for timestamp in metrics[partition][metric_name].keys(): # Timestamp iterator
                if timestamp not in partition_metrics:
                    partition_metrics[timestamp] = {}
                metric = metrics[partition][metric_name][timestamp]
                partition_metrics[timestamp][metric_name] = metric  
        # Write metrics
        *_, last = metric_names
        for timestamp in partition_metrics.keys():
            w.write(timestamp + ',')

            for metric_name in metric_names:
                if partition_metrics[timestamp][metric_name] is None:
                    w.write(',')
                else:
                    w.write(partition_metrics[timestamp][metric_name])
                    if last != metric_name:
                        w.write(',')
            w.write('\n')

Creating gc.csv
Creating gpumedium.csv
Creating gpusmall.csv
Creating gputest.csv
Creating interactive.csv
Creating large.csv
Creating medium.csv
Creating test.csv


In [57]:
partition_metrics

{'1681116900': {'availability': '1.0',
  'avg_alloc_mem': '0.0',
  'avg_allocated_cpus_per_job': '256.0',
  'avg_allocated_nodes_per_job': '1.0',
  'avg_cpus_load_lower': '0.0',
  'avg_cpus_load_upper': '0.06',
  'avg_execution_time_per_job': '30600.0',
  'avg_free_mem_lower': '239358.0',
  'avg_free_mem_upper': '249143.0',
  'avg_job_size_lower': '1.0',
  'avg_job_size_upper': '2.0',
  'avg_maximum_allocated_cpus_per_job': '256.0',
  'avg_maximum_allocated_nodes_per_job': '1.0',
  'avg_memory': '240000.0',
  'avg_minimum_requested_cpus_per_job': '4.0',
  'avg_minimum_requested_memory_per_job': '240000.0',
  'avg_minimum_requested_nodes_per_job': '-1.0',
  'avg_pending_jobs': '0.0',
  'avg_queue_time_per_job': '0.0',
  'avg_requested_cpus_per_job': '-1.0',
  'avg_running_jobs': '1.0',
  'avg_time_left_per_job': '23400.0',
  'avg_time_limit': '3600.0',
  'cores': '16.0',
  'cpus': '256.0',
  'node_alloc': '0.0',
  'node_idle': '8.0',
  'node_other': '0.0',
  'node_total': '8.0',
  'node